# 1. Create the design in Metal

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [3]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [4]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

# 2. 4 qubits + 4 CPWs

In [5]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um',
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        bus1 = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        bus2 = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'bus2', 'Q2', 'bus1', '6.0 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'bus1', 'Q2', 'bus2', '6.1 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'bus2', 'Q4', 'bus1', '6.0 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'bus1', 'Q4', 'bus2', '6.1 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

# 2. Capacitance Analysis and LOM derivation using the analysis package - most users

In [6]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [8]:
# example: update single setting
c1.sim.setup.max_passes = 6
# example: update multiple settings
c1.sim.setup_update(solution_order = 'Medium', auto_increase_solution_order = 'False')

c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': 5.0,
 'save_fields': False,
 'enabled': True,
 'max_passes': 6,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [10]:
c1.sim.run(components=['Q1'], open_terminations=[('Q1', 'readout'), ('Q1', 'bus1'), ('Q1', 'bus2')])
c1.sim.capacitance_matrix

INFO 10:29AM [connect_project]: Connecting to Ansys Desktop API...
INFO 10:29AM [load_ansys_project]: 	Opened Ansys App
INFO 10:29AM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 10:29AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/Airplane/OneDrive - skku.edu/문서/Ansoft/
	Project:   Project1
INFO 10:29AM [connect_design]: No active design found (or error getting active design).
INFO 10:29AM [connect]: 	 Connected to project "Project1". No design detected
INFO 10:29AM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 10:29AM [connect_setup]: 	No design setup detected.
WARNING 10:29AM [connect_setup]: 	Creating Q3D default setup.
INFO 10:29AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 10:29AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 10:29AM [analyze]: Analyzing setup Setup


com_error: (-2147352567, '예외가 발생했습니다.', (0, None, None, None, 0, -2147024349), None)

In [ ]:
c1.sim.close()

In [9]:
c1.sim.start()

INFO 09:26AM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:26AM [load_ansys_project]: 	Opened Ansys App
INFO 09:26AM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 09:26AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/Airplane/OneDrive - skku.edu/문서/Ansoft/
	Project:   Tutorial_CapMtxLOM
INFO 09:26AM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 09:26AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 09:26AM [connect]: 	Connected to project "Tutorial_CapMtxLOM" and design "Design_q3d" 😀 



In [10]:
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 09:26AM [get_matrix]: Exporting matrix data to (C:\Users\Airplane\AppData\Local\Temp\tmptt7e2aj3.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 09:26AM [get_matrix]: Exporting matrix data to (C:\Users\Airplane\AppData\Local\Temp\tmpv08r3l01.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 1, False
INFO 09:26AM [get_matrix]: Exporting matrix data to (C:\Users\Airplane\AppData\Local\Temp\tmpxwwxx4k2.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 2, False
INFO 09:26AM [get_matrix]: Exporting matrix data to (C:\Users\Airplane\AppData\Local\Temp\tmpmby1rjjh.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 3, False
INFO 09:26AM [get_matrix]: Exporting matrix data to (C:\Users\Airplane\AppData\Local\Temp\tmpavvxrdnq.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5000000000, Maxwell, 4, F

,bus1_connector_pad_Q1,bus2_connector_pad_Q1,ground_main_plane,pad_bot_Q1,pad_top_Q1,readout_connector_pad_Q1
bus1_connector_pad_Q1,49.77799,-0.42560,-33.50864,-1.57029,-13.15538,-0.20494
bus2_connector_pad_Q1,-0.42560,54.01891,-35.77524,-13.99741,-1.82852,-1.01320
ground_main_plane,-33.50864,-35.77524,237.69027,-31.53489,-37.88739,-36.55731
pad_bot_Q1,-1.57029,-13.99741,-31.53489,98.20664,-30.07382,-18.86490
pad_top_Q1,-13.15538,-1.82852,-37.88739,-30.07382,87.85084,-2.20122
readout_connector_pad_Q1,-0.20494,-1.01320,-36.55731,-18.86490,-2.20122,59.92351


In [11]:
type(c1.sim.capacitance_matrix)

pandas.core.frame.DataFrame

# Lumped oscillator model (LOM)

In [12]:
c1.setup.junctions = Dict({'Lj': 12.31, 'Cj': 2})
c1.setup.freq_readout = 7.0
c1.setup.freq_bus = [6.0, 6.2]

c1.run_lom()
c1.lumped_oscillator_all

[3, 4] [5 0 1]
Predicted Values

Transmon Properties
f_Q 5.424935 [GHz]
EC 311.976948 [MHz]
EJ 13.273404 [GHz]
alpha -363.792406 [MHz]
dispersion 46.550353 [KHz]
Lq 12.305036 [nH]
Cq 62.088652 [fF]
T1 35.336801 [us]

**Coupling Properties**

tCqbus1 7.383745 [fF]
gbus1_in_MHz 114.265711 [MHz]
χ_bus1 -3.174186 [MHz]
1/T1bus1 2809.273326 [Hz]
T1bus1 56.653421 [us]

tCqbus2 -6.455284 [fF]
gbus2_in_MHz -85.831848 [MHz]
χ_bus2 -9.970477 [MHz]
1/T1bus2 1205.470295 [Hz]
T1bus2 132.027262 [us]

tCqbus3 5.372188 [fF]
gbus3_in_MHz 73.765124 [MHz]
χ_bus3 -4.515410 [MHz]
1/T1bus3 489.199561 [Hz]
T1bus3 325.337461 [us]
Bus-Bus Couplings
gbus1_2 7.097122 [MHz]
gbus1_3 9.957520 [MHz]
gbus2_3 5.377145 [MHz]


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.748104,353.230672,13.273404,-417.375147,135.208268,"[108.79200978040816, -73.21708181328188, 76.45...","[-4.790610087146361, -26.577069875285524, -12....",4.790610,108.792010
2,5.664673,342.29546,13.273404,-403.045756,103.898361,"[112.4840344536537, -82.16155872148896, 68.802...","[-4.459428523665049, -22.01884264095459, -7.62...",4.459429,112.484034
3,5.574019,330.639508,13.273404,-387.872696,77.330632,"[111.33611899021486, -84.09226364022662, 71.77...","[-3.7804641524649494, -15.86558554786528, -6.3...",3.780464,111.336119
4,5.523032,324.186357,13.273404,-379.516629,65.212167,"[110.99790102064628, -84.10822360591217, 72.87...","[-3.472071661530782, -13.18585634948887, -5.66...",3.472072,110.997901
5,5.463771,316.778216,13.273404,-369.962558,53.275079,"[113.07913083130138, -84.87715341728405, 72.79...","[-3.293717610482083, -11.011720591398394, -4.8...",3.293718,113.079131
6,5.424935,311.976948,13.273404,-363.792406,46.550353,"[114.26571069942564, -85.83184768292763, 73.76...","[-3.174186233473547, -9.970476953012021, -4.51...",3.174186,114.265711


In [13]:
c1.plot_convergence();
c1.plot_convergence_chi()
# -> g,chk convergence vs pass

INFO 09:27AM [hfss_report_full_convergence]: Creating report for variation 0


Design "Design_q3d" info:
	# eigenmodes    0
	# variations    1


In [14]:
c1.sim.close()